In [1]:
# Minigpt Vit module
#   (visual_encoder): VisionTransformer(
#     (patch_embed): PatchEmbed(
#       (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
#     )
#     (pos_drop): Dropout(p=0.0, inplace=False)
#     (blocks): ModuleList(
#       (0-38): 39 x Block(
#         (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
#         (attn): Attention(
#           (qkv): Linear(in_features=1408, out_features=4224, bias=False)
#           (attn_drop): Dropout(p=0.0, inplace=False)
#           (proj): Linear(in_features=1408, out_features=1408, bias=True)
#           (proj_drop): Dropout(p=0.0, inplace=False)
#         )
#         (drop_path): Identity()
#         (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
#         (mlp): Mlp(
#           (fc1): Linear(in_features=1408, out_features=6144, bias=True)
#           (act): GELU(approximate='none')
#           (fc2): Linear(in_features=6144, out_features=1408, bias=True)
#           (drop): Dropout(p=0.0, inplace=False)
#         )
#       )
#     )
#   )
#   (ln_vision): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
#   (llama_proj): Linear(in_features=5632, out_features=4096, bias=True)
# )

In [2]:
import glob, os,sys
import torch, numpy as np
import torch.nn as nn
import torch.nn.functional as F
from functools import partial
from transformers import AutoModel, AutoConfig
from model.eva_vit import create_eva_vit_g,VisionTransformer
from PIL import Image

In [3]:
from transformers import BlipProcessor, BlipForQuestionAnswering
from transformers import CLIPProcessor, CLIPModel
# processor = BlipProcessor.from_pretrained("../../VITModel/blip-vqa-base/")
# model = BlipForQuestionAnswering.from_pretrained("../../VITModel/blip-vqa-base/", torch_dtype=torch.float16).to("cuda")
# model = CLIPModel.from_pretrained("../../VITModel/clip-vit-base-patch32", torch_dtype=torch.float16).to("cuda")
# model = CLIPModel.from_pretrained("../../VITModel/clip-vit-large-patch14-336", torch_dtype=torch.float16).to("cuda")
# processor = CLIPProcessor.from_pretrained("../../VITModel/clip-vit-base-patch32")
# processor = CLIPProcessor.from_pretrained("../../VITModel/clip-vit-large-patch14-336")

In [4]:
from utils.caption_datasets import COCOCaptionDataset
from torch.utils.data import DataLoader
# demo測試對話反饋
model_config = {'amp':True, 'use_distributed':False,'accum_grad_iters':1,
                'chat_template': True, 'end_sym': '\n', 'prompt_template': "<s>[INST] {} [/INST]",
                'max_txt_len': 1024, 'max_context_len': 3500,
                'ouput_dir': './exper01_llama2',
                'stage_ckpt': '/ssd3/chih/LLM/MiniGPT-4-ckpt/checkpoint_stage3.pth', 
                'vis_root_train': './dataset/minigpt_casing_train/coco/image/train',
                'ann_paths_train': ['./dataset/minigpt_casing_train/coco_caption/defe_ready_anno.json'],
                'vis_root_valid': './dataset/minigpt_casing_test/coco/image/test',
                'ann_paths_valid': ['./dataset/minigpt_casing_test/coco_caption/defe_ready_anno.json']}
train_data_set = COCOCaptionDataset(vis_root=model_config['vis_root_valid'], 
                                ann_paths=model_config['ann_paths_valid'],
                                img_size=336)
train_dataloader =DataLoader(train_data_set, batch_size=2, num_workers=10, shuffle=True, pin_memory=True)

ann paths ['./dataset/minigpt_casing_test/coco_caption/defe_ready_anno.json']
{'caption': 'Defective.', 'image': '/ssd3/chih/Dataset/minigpt_casing_test/coco/image/test/casting512x512_deffront_0.jpeg', 'image_id': 'casting512x512_deffront_0.jpeg', 'instance_id': '0'}
./dataset/minigpt_casing_test/coco/image/test/casting512x512_okfront_1169.jpeg 



In [5]:
for i in train_dataloader:
    break

In [8]:
class VisionTransformer(nn.Module):
    def __init__(self, model_path = None, img_size=224, patch_size=16, in_chans=3, num_classes=1000, embed_dim=1024):
        super().__init__()
        self.image_size = img_size
        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models
        self.forward_encoder = CLIPModel.from_pretrained(model_path, torch_dtype=torch.float16).vision_model
    def forward(self, x):
        x = self.forward_encoder(x)
        if len(x)!=1: #過濾掉clip中post層和自帶的layernorm
            x = x[0]
        return x
model = VisionTransformer(model_path="../../VITModel/clip-vit-large-patch14", img_size=336)
model

VisionTransformer(
  (forward_encoder): CLIPVisionTransformer(
    (embeddings): CLIPVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (position_embedding): Embedding(257, 1024)
    )
    (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-23): 24 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, 

In [45]:
# image = Image.open("/ssd3/chih/branch/VLM_Rebuild/dataset/minigpt_casing_train/coco/image/train/casting512x512_deffront_0.jpeg")
# inputs = processor(images=image, return_tensors="pt", padding=True)
# image = Image.open("/ssd3/chih/branch/VLM_Rebuild/000000039769.jpg")
# inputs = processor(images=image, return_tensors="pt", padding=True)
# inputs['pixel_values'].cuda().shape

In [ ]:
# model_v2 = VisionTransformer(
#     img_size=448,
#     patch_size=14,
#     use_mean_pooling=False,
#     embed_dim=1408,
#     depth=39,
#     num_heads=1408//88,
#     mlp_ratio=4.3637,
#     qkv_bias=True,
#     drop_path_rate=0,
#     norm_layer=partial(nn.LayerNorm, eps=1e-6),
#     # use_checkpoint=use_checkpoint,
# )  

In [21]:
# v2_output = model_v2(i['image'])
# i['image'].cuda().shape